# MissOh DataLoader

In [505]:
# authored by haeyong.kang
# date : 2020/06/21

### AnotherMissOh Visual Structure
- json_data['file_name'] : 'AnotherMissOh01.mp4'
- json_data['visual_results']
- json_data['visual_results'][0].keys() : dict_keys(['start_time', 'end_time', 'vid', 'image_info'])
- {
'start_time': '00:02:51;16', 
'end_time': '00:02:54;15', 
'vid': 'AnotherMissOh01_001_0078', 
'image_info': ...}
- json_data['visual_results'][0]['image_info']
- [{'frame_id': 'AnotherMissOh01_001_0078_IMAGE_0000004295', 
'place': 'none', 
'persons': [
{'person_id': 'Haeyoung1', 
'person_info': {
'face_rect': {'min_x': 515, 'min_y': 0, 'max_x': 845, 'max_y': 443}, 
'full_rect': {'min_x': 278, 'min_y': 2, 'max_x': 1025, 'max_y': 769}, 
'behavior': 'stand up', 
'predicate': 'none', 
'emotion': 'Neutral', 
'face_rect_score': '0.5', 
'full_rect_score': '0.9'}, 
'related_objects': []}], 
'objects': []}, 
- {'frame_id': 'AnotherMissOh01_001_0078_IMAGE_0000004311', 
'place': '', 
'persons': [{
'person_id':'Haeyoung1',
'person_info': {
'face_rect': {'min_x': 515, 'min_y': 0, 'max_x': 831, 'max_y': 411}, 
'full_rect': {'min_x': 270, 'min_y': 0, 'max_x': 1025, 'max_y': 768}, 
'behavior': 'stand up', 
'predicate': 'none', 
'emotion': 'Neutral', 
'face_rect_score': '0.5', 
'full_rect_score': '0.9'}, 
'related_objects': []}],
'objects': []},]

In [506]:
import sys
sys.path.append("../") # go to parent dir

In [507]:
import os
from torch.utils.data import Dataset, DataLoader
import cv2
import pickle
import numpy as np
import glob
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import json
import argparse
import matplotlib.pyplot as plt

from Yolo_v2_pytorch.src.utils import *
from graphviz import Digraph, Graph

import networkx as nx
from networkx.drawing.nx_pydot import read_dot
#from networkx.drawing.nx_agraph import read_dot
from networkx.readwrite import json_graph

In [508]:
read_dot

<function networkx.drawing.nx_pydot.read_dot(path)>

In [509]:
def is_not_blank(s):
    return bool(s and s.strip())

In [510]:
MissOh_CLASSES = ['person']
print(MissOh_CLASSES[0])
global colors
colors = pickle.load(open("../Yolo_v2_pytorch/src/pallete", "rb"))
print(colors[0])

person
(39, 129, 113)


In [511]:
def get_args():
    parser = argparse.ArgumentParser(
        "You Only Look Once:Unified, Real-Time Object Detection")
    parser.add_argument("--image_size", type=int,
                        default=448,
                        help="The common width and height for all images")
    parser.add_argument("--batch_size", type=int, default=1,
                        help="The number of images per batch")
    # Training base Setting
    parser.add_argument("--momentum", type=float, default=0.9)
    parser.add_argument("--decay", type=float, default=0.0005)
    parser.add_argument("--dropout", type=float, default=0.5)
    parser.add_argument("--num_epoches", type=int, default=100)
    parser.add_argument("--test_interval", type=int, default=1,
                        help="Number of epoches between testing phases")
    parser.add_argument("--object_scale", type=float, default=1.0)
    parser.add_argument("--noobject_scale", type=float, default=0.5)
    parser.add_argument("--class_scale", type=float, default=1.0)
    parser.add_argument("--coord_scale", type=float, default=5.0)
    parser.add_argument("--reduction", type=int, default=32)
    parser.add_argument("--es_min_delta", type=float, default=0.0,
                        help="Early stopping's parameter:minimum change loss to qualify as an improvement")
    parser.add_argument("--es_patience", type=int, default=0,
                        help="Early stopping's parameter:number of epochs with no improvement after which training will be stopped. Set to 0 to disable this technique.")

    parser.add_argument("--pre_trained_model_type",
                        type=str, choices=["model", "params"],
                        default="model")
    parser.add_argument("--pre_trained_model_path", type=str,
                        default="Yolo_v2_pytorch/trained_models/only_params_trained_yolo_voc") # Pre-training path

    parser.add_argument("--saved_path", type=str,
                        default="./checkpoint") # saved training path
    parser.add_argument("--conf_threshold", type=float, default=0.35)
    parser.add_argument("--nms_threshold", type=float, default=0.5)
    args = parser.parse_args(args=[]) # for jupyter 
    return args

In [512]:
opt = get_args()
print(opt)

Namespace(batch_size=1, class_scale=1.0, conf_threshold=0.35, coord_scale=5.0, decay=0.0005, dropout=0.5, es_min_delta=0.0, es_patience=0, image_size=448, momentum=0.9, nms_threshold=0.5, noobject_scale=0.5, num_epoches=100, object_scale=1.0, pre_trained_model_path='Yolo_v2_pytorch/trained_models/only_params_trained_yolo_voc', pre_trained_model_type='model', reduction=32, saved_path='./checkpoint', test_interval=1)


In [513]:
# visualize the images and labels
height, width = (768, 1024)
width_ratio = 448 / width
height_ratio = 448 / height

In [514]:
class AnotherMissOh(Dataset):
    def __init__(self, dataset, img_path, json_path, display_log=True):
        
        self.display_log = display_log
        self.init_clips(img_path)
        self.load_json(dataset,img_path, json_path)
        
    def init_clips(self, img_path):
        self.cnt_clips = 0
        self.img_path = img_path
        
        self.img_size = (1024, 768)
        self.img_scaled_size = (448, 448)
        
        tform = [
            Resize(self.img_scaled_size), # should match to Yolo_V2
            ToTensor(), 
            #Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # should match to Yolo_V2
        ]
        self.transformations = Compose(tform)
        
        '''
        clips = {
            'episode' : [],
            'clip' : [],
            'start_time' : [],
            'end_time' : [],
            'vid' : [],
            'img_size' : [],
            'img_scaled_size' : [],
            'image_info' : []}
            
        image_info = {
            'frame_id': [],
            'place' : [],
            'persons' : []}
            
        persons = {
            'person_id': [],
            'face_rect' : [],
            'full_rect' : [],
            'behavior' : [],
            'predicate' : [],
            'emotion' : [],
            'face_rect_score' : [],
            'full_rect_score' : []}
        '''
        
    def load_json(self, dataset, img_path, json_path):
        
        self.clips = []
        
        for episode in dataset:
            img_dir = img_path + 'AnotherMissOh{:02}/'.format(episode)
            json_dir = json_path + 'AnotherMissOh{:02}_visual.json'.format(episode)
            if self.display_log:
                print('imag_dir:{}'.format(img_dir))
                print('json_dir:{}'.format(json_dir))

            with open(json_dir, encoding='utf-8') as json_file:
                json_data = json.load(json_file)

            for i in range(len(json_data['visual_results'])):
                clip = {}
                clip['episode'] = []
                clip['clip'] = []
                clip['start_time'] = []
                clip['end_time'] = []
                clip['vid'] = []
                clip['image_info'] = []
                
                if self.display_log:
                    print("***{}th episode***{}th clips***************************************".format(episode, i))
                    print("['visual_results'][{}]['start_time']:{}".format(i,json_data['visual_results'][i]['start_time']))
                    print("['visual_results'][{}]['end_time']:{}".format(i,json_data['visual_results'][i]['end_time']))
                    print("['visual_results'][{}]['vid']:{}".format(i,json_data['visual_results'][i]['vid'].replace('_', '/')))
                    print("['visual_results'][{}]['img_size']:{}".format(i,img_size))
                    print("['visual_results'][{}]['img_scaled_size']:{}".format(i,img_scaled_size))
                    print("['visual_results'][{}]['episode']:{}".format(i,episode))
                
                clip['episode'].append(episode)
                clip['clip'].append(i)
                clip['start_time'].append(json_data['visual_results'][i]['start_time'])
                clip['end_time'].append(json_data['visual_results'][i]['end_time'])
                clip['vid'].append(json_data['visual_results'][i]['vid'].replace('_', '/'))
                
                
                for j, info in enumerate(json_data['visual_results'][i]['image_info']):
                    image_info = {}
                    image_info['frame_id'] = []
                    image_info['place'] = []
                    image_info['persons'] = {}
                    
                    if self.display_log:
                        print("=============={}th frame========================================".format(j))
                        
                    img_file = img_dir + json_data['visual_results'][i]['vid'].replace('_', '/')[-8:] + '/'+ info['frame_id'][-16:] + '.jpg'
                    image_info['frame_id'].append(img_file)
                    image_info['place'].append(info['place'])
                    
                    image_info['persons']['person_id']=[]
                    image_info['persons']['face_rect']=[]
                    image_info['persons']['full_rect']=[]
                    image_info['persons']['behavior']=[]
                    image_info['persons']['predicate']=[]
                    image_info['persons']['emotion']=[]
                    image_info['persons']['face_rect_score']=[]
                    image_info['persons']['full_rect_score']=[]
                    for k, person in enumerate(info['persons']):
                        if self.display_log:
                            print("--------------------{}th person-----------------------------".format(k))
                            
                        image_info['persons']['person_id'].append(person['person_id'])
                        face_bbox = person['person_info']['face_rect']
                        if (face_bbox['min_y'] == "" 
                            or face_bbox['max_y'] == "" 
                            or face_bbox['min_x'] == "" 
                            or face_bbox['max_x'] == ""):
                            face_rect = []
                            continue
                        else:
                            face_rect = [face_bbox['min_x'], face_bbox['min_y'], face_bbox['max_x'], face_bbox['max_y']]
                        image_info['persons']['face_rect'].append(face_rect)
                        full_bbox = person['person_info']['full_rect']
                        if (full_bbox['min_y'] == "" 
                            or full_bbox['max_y'] == "" 
                            or full_bbox['min_x'] == "" 
                            or full_bbox['max_x'] == ""):
                            full_rect = []
                            continue
                        else:
                            full_rect = [full_bbox['min_x'], full_bbox['min_y'], full_bbox['max_x'], full_bbox['max_y']]
                        image_info['persons']['full_rect'].append(full_rect)
                        image_info['persons']['behavior'].append(person['person_info']['behavior'])
                        image_info['persons']['predicate'].append(person['person_info']['predicate'])
                        image_info['persons']['emotion'].append(person['person_info']['emotion'])
                        image_info['persons']['face_rect_score'].append(person['person_info']['face_rect_score'])
                        image_info['persons']['full_rect_score'].append(person['person_info']['full_rect_score'])
                        
                    clip['image_info'].append(image_info)
                self.clips.append(clip)
                    
    def __len__(self):
        return len(self.clips)
                    
    def __getitem__(self, item):
        info = self.clips[item]['image_info']
        episode = self.clips[item]['episode']
        clip = self.clips[item]['clip']
        start_time = self.clips[item]['start_time']
        end_time = self.clips[item]['start_time']
        
        images = []
        for it, frame in enumerate(info):
            img = Image.open(frame['frame_id'][0]).convert('RGB')
            img = self.transformations(img)
            images.append(img)
        
        return images, info, episode, clip, start_time, end_time

In [515]:
img_path = '../data/AnotherMissOh/AnotherMissOh_images/'
json_path = '../data/AnotherMissOh/AnotherMissOh_Visual/'

episode = 1
train = [episode]
train_set = AnotherMissOh(train, img_path, json_path, False)

In [532]:
def graph(episode, scene, frm,st,et,info, save_file, debug = False):
    import string
    strseq = string.ascii_uppercase
    
    # define  graph
    dot = Digraph('G',filename='{}.gv'.format(save_file),engine='fdp')
    dot.attr('graph', rotate = '0', dpi='600',rankdir='TB', size='10,5')
    dot.attr('node', height='0.1', fontsize='6')
    dot.attr('edge', fontsize='5')

    place = "{}".format(info['place'][0])
    sound = "{}".format('sound')
    
    if not is_not_blank(place):
        place = 'none'
    if not is_not_blank(sound):
        sound = 'none'
        
    num_of_persons = len(info['persons']['person_id'])
    
    frm_graph = 'episode_{}_scene_{}_frame_{}'.format(
        episode, scene, frm)
    
    #dot.node(frm_graph, style='filled', color='lightgrey')
    episode_node = "episode_{:02d}".format(episode)
    scene_node = "scene_{:03d}".format(scene)
    frame_node = "frame_{:04d}".format(frm)
    dot.node(episode_node, style='filled', color='lightgrey')
    dot.node(scene_node, style='filled', color='lightgrey')
    dot.node(frame_node, style='filled', color='lightgrey')
    
    dot.node(place, style='filled', color='lightblue')
    dot.node(sound, style='filled', color='lightblue')
    
    if is_not_blank(episode_node) and is_not_blank(scene_node):
        dot.edge(episode_node, scene_node)
    
    if is_not_blank(scene_node) and is_not_blank(frame_node):
        dot.edge(scene_node, frame_node)
        
    if is_not_blank(frame_node) and is_not_blank(place):
        dot.edge(frame_node, place)
    
    if is_not_blank(frame_node) and is_not_blank(sound):
        dot.edge(frame_node, sound)

    for p in range(num_of_persons):
        
        try:
            person_id = "{}".format(info['persons']['person_id'][p])
        except:
            person_id = 'none'
            continue
        try:
            behavior = "{}".format(info['persons']['behavior'][p])
        except:
            person_id = 'none'
            continue
        try:
            predicate = "{}".format(info['persons']['predicate'][p])
        except:
            person_id = 'none'
            continue
        try:
            emotion = "{}".format(info['persons']['emotion'][p])
        except:
            person_id = 'none'
            continue

        if is_not_blank(person_id):
            dot.node(person_id)
        if is_not_blank(behavior):
            dot.node(behavior, style='filled', color='green')
        if is_not_blank(predicate):
            dot.node(predicate, style='filled', color='yellow')
        if is_not_blank(emotion):
            dot.node(emotion, style='filled', color='blue')

        if is_not_blank(frame_node) and is_not_blank(person_id):
            dot.edge(frame_node, person_id)
        if is_not_blank(person_id) and is_not_blank(behavior):
            dot.edge(person_id, behavior)
        if is_not_blank(person_id) and is_not_blank(predicate):
            dot.edge(person_id, predicate)
        if is_not_blank(person_id) and is_not_blank(emotion):
            dot.edge(person_id, emotion)
            
        
            
    # show in image
    dot.format = 'png'
    dot.render('{}.gv'.format(save_file), view=True)

    graph = cv2.imread('{}.gv.png'.format(save_file))
    graph = cv2.resize(graph, dsize=(0, 0), fx=600.0/graph.shape[0], fy=600.0/graph.shape[0])

    if debug:
        plt.figure(figsize=(8,8))
        plt.imshow(graph)
        plt.show()

In [533]:
save_dir = '../results/drama_graph/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
debug = False

for scene in range(len(train_set)):
    
    images, info, episode, scene, st, et = train_set[scene]
    scene= scene[0]
    episode= episode[0]

    for frm in range(len(images)):
        image = images[frm].cpu().numpy()
        #print(image)
        imageInfo = cv2.cvtColor(np.transpose(image * 255,(1,2,0)), 
                                 cv2.COLOR_RGB2BGR)

        frm_name = "episode_{:02d}_scene_{:03d}_frame_{:04d}".format(episode, scene, frm)
        save_file = save_dir + frm_name
        print("episode:{}, scene:{}, frame:{} st:{}, et:{}".format(
            episode, scene, frm, st, et))

        place = info[frm]['place'][0]
        sound = 'sound'
        num_of_persons = len(info[frm]['persons']['person_id'])
        graph(episode, scene, frm, st, et ,info[frm], save_file, debug)

        # read dot graph
        dot_graph = nx.nx_pydot.read_dot('{}.gv'.format(save_file))
        s_graph = json.dumps(json_graph.node_link_data(dot_graph))

        for p in range(num_of_persons):
            try: 
                person_id = info[frm]['persons']['person_id'][p]
            except:
                person_id = 'none'
                continue
            try:
                face_rect= info[frm]['persons']['face_rect'][p]
            except:
                face_rect = 'none'
                continue
            try: 
                full_rect = info[frm]['persons']['full_rect'][p]
            except:
                full_rect = 'none'
                continue
            try: 
                behavior = info[frm]['persons']['behavior'][p]
            except:
                behavior = 'none'
                continue
            try:
                predicate = info[frm]['persons']['predicate'][p]
            except:
                predicate = 'none'
                continue
            try: 
                emotion = info[frm]['persons']['emotion'][p]
            except:
                emotion = 'none'
                continue

            print("place:{}".format(place))
            print("num_of_persons:{}".format(num_of_persons))
            print("person_id:{}".format(person_id))
            print("face_rect:{}".format(face_rect))
            print("full_rect:{}".format(full_rect))
            print("behavior:{}".format(behavior))
            print("predicate:{}".format(predicate))
            print("emotion:{}".format(emotion))

            # face rect
            xmin = int(max(face_rect[0] * width_ratio, 0))
            ymin = int(max(face_rect[1] * height_ratio, 0))
            xmax = int(min((face_rect[2]) * width_ratio, 448))
            ymax = int(min((face_rect[3]) * height_ratio, 448))
            cv2.rectangle(imageInfo, (xmin, ymin), (xmax, ymax), colors[0], 2)

            # full rect
            xmin = int(max(full_rect[0] * width_ratio, 0))
            ymin = int(max(full_rect[1] * height_ratio, 0))
            xmax = int(min((full_rect[2]) * width_ratio, 448))
            ymax = int(min((full_rect[3]) * height_ratio, 448))
            cv2.rectangle(imageInfo, (xmin, ymin), (xmax, ymax), colors[2], 2)
            cv2.putText(imageInfo, person_id, (xmin, ymin), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (255.0, 255.0, 255.0), 1)
            cv2.putText(imageInfo, emotion, (xmin, ymax+20), 
                        cv2.FONT_HERSHEY_PLAIN, 1, (0.0, 0.0, 255.0), 1)
            cv2.putText(imageInfo, behavior, (xmin, ymax+30), 
                        cv2.FONT_HERSHEY_PLAIN, 1, (0.0, 255.0, 0.0), 1)
            cv2.putText(imageInfo, place, (10, 20), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (255.0, 255.0, 255.0), 1)
            cv2.putText(imageInfo, sound, (10, 40), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (255.0, 255.0, 255.0), 1)

        imageInfo = cv2.resize(imageInfo,dsize=(1024, 768))
        cv2.imwrite("{}_frame.png".format(save_file), imageInfo)
        imageInfo = cv2.cvtColor(imageInfo, cv2.COLOR_BGR2RGB)

        if debug:
            plt.figure(figsize=(8,8))
            plt.imshow(np.uint8(imageInfo))
            plt.show()

episode:1, scene:0, frame:0 st:['00:02:51;16'], et:['00:02:51;16']
place:none
num_of_persons:1
person_id:Haeyoung1
face_rect:[515, 0, 845, 443]
full_rect:[278, 2, 1025, 769]
behavior:stand up
predicate:none
emotion:Neutral
episode:1, scene:0, frame:1 st:['00:02:51;16'], et:['00:02:51;16']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[515, 0, 831, 411]
full_rect:[270, 0, 1025, 768]
behavior:stand up
predicate:none
emotion:Neutral
episode:1, scene:0, frame:2 st:['00:02:51;16'], et:['00:02:51;16']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[515, 0, 829, 442]
full_rect:[255, 0, 1022, 767]
behavior:stand up
predicate:none
emotion:Neutral
episode:1, scene:0, frame:3 st:['00:02:51;16'], et:['00:02:51;16']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[514, 0, 836, 438]
full_rect:[255, 0, 1022, 767]
behavior:stand up
predicate:none
emotion:Neutral
episode:1, scene:0, frame:4 st:['00:02:51;16'], et:['00:02:51;16']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect

place:kitchen
num_of_persons:2
person_id:Jeongsuk
face_rect:[463, 97, 523, 182]
full_rect:[402, 98, 547, 586]
behavior:eat
predicate:
emotion:Happiness
place:kitchen
num_of_persons:2
person_id:Deogi
face_rect:[236, 85, 334, 170]
full_rect:[235, 74, 361, 603]
behavior:stand up
predicate:
emotion:Happiness
episode:1, scene:6, frame:4 st:['00:03:06;03'], et:['00:03:06;03']
place:kitchen
num_of_persons:2
person_id:Jeongsuk
face_rect:[463, 102, 523, 187]
full_rect:[402, 98, 547, 586]
behavior:eat
predicate:
emotion:Happiness
place:kitchen
num_of_persons:2
person_id:Deogi
face_rect:[236, 85, 334, 170]
full_rect:[235, 74, 361, 603]
behavior:stand up
predicate:
emotion:Happiness
episode:1, scene:6, frame:5 st:['00:03:06;03'], et:['00:03:06;03']
episode:1, scene:6, frame:6 st:['00:03:06;03'], et:['00:03:06;03']
episode:1, scene:7, frame:0 st:['00:03:08;07'], et:['00:03:08;07']
episode:1, scene:7, frame:1 st:['00:03:08;07'], et:['00:03:08;07']
episode:1, scene:7, frame:2 st:['00:03:08;07'], et:[

place:livingroom
num_of_persons:2
person_id:Kyungsu
face_rect:[656, 390, 711, 460]
full_rect:[624, 336, 789, 606]
behavior:drink
predicate:
emotion:Happiness
place:livingroom
num_of_persons:2
person_id:Jeongsuk
face_rect:[609, 165, 651, 217]
full_rect:[572, 150, 676, 328]
behavior:cook
predicate:
emotion:Happiness
episode:1, scene:11, frame:18 st:['00:03:13;15'], et:['00:03:13;15']
place:livingroom
num_of_persons:2
person_id:Jeongsuk
face_rect:[608, 155, 650, 219]
full_rect:[561, 160, 673, 320]
behavior:cook
predicate:
emotion:Happiness
place:livingroom
num_of_persons:2
person_id:Kyungsu
face_rect:[634, 406, 684, 471]
full_rect:[592, 341, 770, 609]
behavior:drink
predicate:
emotion:Happiness
episode:1, scene:11, frame:19 st:['00:03:13;15'], et:['00:03:13;15']
place:livingroom
num_of_persons:2
person_id:Kyungsu
face_rect:[624, 402, 679, 472]
full_rect:[597, 354, 747, 602]
behavior:drink
predicate:
emotion:Happiness
place:livingroom
num_of_persons:2
person_id:Jeongsuk
face_rect:[586, 158

place:livingroom
num_of_persons:2
person_id:Kyungsu
face_rect:[174, 206, 273, 346]
full_rect:[104, 126, 420, 763]
behavior:none
predicate:
emotion:Happiness
episode:1, scene:16, frame:5 st:['00:03:30;04'], et:['00:03:30;04']
place:livingroom
num_of_persons:2
person_id:Kyungsu
face_rect:[183, 231, 281, 357]
full_rect:[79, 177, 423, 763]
behavior:none
predicate:
emotion:Happiness
place:livingroom
num_of_persons:2
person_id:Deogi
face_rect:[685, 174, 749, 280]
full_rect:[629, 104, 836, 557]
behavior:none
predicate:
emotion:Happiness
episode:1, scene:17, frame:0 st:['00:03:32;00'], et:['00:03:32;00']
place:livingroom
num_of_persons:1
person_id:Haeyoung1
face_rect:[524, 23, 826, 443]
full_rect:[401, 0, 998, 766]
behavior:none
predicate:
emotion:Anger
episode:1, scene:17, frame:1 st:['00:03:32;00'], et:['00:03:32;00']
episode:1, scene:17, frame:2 st:['00:03:32;00'], et:['00:03:32;00']
place:livingroom
num_of_persons:1
person_id:Haeyoung1
face_rect:[514, 12, 830, 446]
full_rect:[387, 0, 1006,

place:livingroom
num_of_persons:1
person_id:Haeyoung1
face_rect:[118, 277, 195, 381]
full_rect:[47, 172, 287, 645]
behavior:sit down
predicate:
emotion:Anger
episode:1, scene:17, frame:31 st:['00:03:32;00'], et:['00:03:32;00']
place:livingroom
num_of_persons:1
person_id:Haeyoung1
face_rect:[123, 279, 194, 378]
full_rect:[63, 192, 289, 631]
behavior:sit down
predicate:
emotion:Anger
episode:1, scene:17, frame:32 st:['00:03:32;00'], et:['00:03:32;00']
place:livingroom
num_of_persons:1
person_id:Haeyoung1
face_rect:[131, 279, 205, 383]
full_rect:[65, 206, 299, 636]
behavior:none
predicate:
emotion:Anger
episode:1, scene:17, frame:33 st:['00:03:32;00'], et:['00:03:32;00']
place:livingroom
num_of_persons:1
person_id:Haeyoung1
face_rect:[136, 282, 205, 380]
full_rect:[92, 174, 289, 659]
behavior:none
predicate:
emotion:Anger
episode:1, scene:17, frame:34 st:['00:03:32;00'], et:['00:03:32;00']
place:livingroom
num_of_persons:1
person_id:Haeyoung1
face_rect:[136, 279, 213, 381]
full_rect:[135,

...

place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[636, 134, 678, 196]
full_rect:[576, 71, 760, 742]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:205, frame:6 st:['00:11:28;11'], et:['00:11:28;11']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[652, 134, 694, 196]
full_rect:[576, 73, 760, 744]
behavior:stand up
predicate:
emotion:Neutral
episode:1, scene:205, frame:7 st:['00:11:28;11'], et:['00:11:28;11']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[663, 136, 705, 198]
full_rect:[586, 73, 770, 744]
behavior:stand up
predicate:
emotion:Neutral
episode:1, scene:205, frame:8 st:['00:11:28;11'], et:['00:11:28;11']
episode:1, scene:205, frame:9 st:['00:11:28;11'], et:['00:11:28;11']
episode:1, scene:205, frame:10 st:['00:11:28;11'], et:['00:11:28;11']
episode:1, scene:205, frame:11 st:['00:11:28;11'], et:['00:11:28;11']
episode:1, scene:205, frame:12 st:['00:11:28;11'], et:['00:11:28;11']
episode:1, scene:205, frame:13 st:['00:11:28;11'],

place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[499, 97, 704, 373]
full_rect:[476, 0, 939, 744]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:206, frame:24 st:['00:11:34;11'], et:['00:11:34;11']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[502, 100, 707, 376]
full_rect:[476, 0, 939, 744]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:206, frame:25 st:['00:11:34;11'], et:['00:11:34;11']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[508, 99, 713, 375]
full_rect:[476, 0, 939, 744]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:206, frame:26 st:['00:11:34;11'], et:['00:11:34;11']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[508, 99, 713, 375]
full_rect:[473, 2, 936, 746]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:206, frame:27 st:['00:11:34;11'], et:['00:11:34;11']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[500, 100, 705, 376]
full_rect:[473, 2, 936, 746]
behavior:

place:office
num_of_persons:1
person_id:Sukyung
face_rect:[170, 192, 438, 583]
full_rect:[109, 2, 724, 767]
behavior:none
predicate:
emotion:Anger
episode:1, scene:213, frame:16 st:['00:11:49;14'], et:['00:11:49;14']
place:office
num_of_persons:1
person_id:Sukyung
face_rect:[231, 183, 499, 574]
full_rect:[159, 2, 774, 767]
behavior:none
predicate:
emotion:Anger
episode:1, scene:213, frame:17 st:['00:11:49;14'], et:['00:11:49;14']
place:office
num_of_persons:1
person_id:Sukyung
face_rect:[259, 181, 527, 572]
full_rect:[196, 2, 811, 767]
behavior:none
predicate:
emotion:Anger
episode:1, scene:213, frame:18 st:['00:11:49;14'], et:['00:11:49;14']
place:office
num_of_persons:1
person_id:Sukyung
face_rect:[288, 183, 556, 574]
full_rect:[213, 2, 828, 767]
behavior:none
predicate:
emotion:Anger
episode:1, scene:213, frame:19 st:['00:11:49;14'], et:['00:11:49;14']
place:office
num_of_persons:1
person_id:Sukyung
face_rect:[279, 179, 547, 570]
full_rect:[199, 2, 814, 767]
behavior:none
predicate:

place:office
num_of_persons:1
person_id:Sukyung
face_rect:[321, 118, 657, 607]
full_rect:[202, 80, 762, 764]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:218, frame:1 st:['00:12:11;11'], et:['00:12:11;11']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[563, 110, 609, 183]
full_rect:[429, 73, 647, 746]
behavior:none
predicate:
emotion:Anger
episode:1, scene:219, frame:0 st:['00:12:12;12'], et:['00:12:12;12']
place:office
num_of_persons:2
person_id:Haeyoung1
face_rect:[362, 144, 509, 409]
full_rect:[2, 5, 554, 744]
behavior:none
predicate:
emotion:Anger
place:office
num_of_persons:2
person_id:Sukyung
face_rect:[713, 120, 859, 349]
full_rect:[678, 0, 1003, 768]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:219, frame:1 st:['00:12:12;12'], et:['00:12:12;12']
place:office
num_of_persons:2
person_id:Haeyoung1
face_rect:[367, 146, 514, 411]
full_rect:[12, 26, 564, 765]
behavior:none
predicate:
emotion:Anger
place:office
num_of_persons:2
person_id:Sukyung

place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[205, 70, 549, 554]
full_rect:[21, 0, 1018, 758]
behavior:none
predicate:
emotion:Anger
episode:1, scene:224, frame:4 st:['00:12:22;00'], et:['00:12:22;00']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[136, 104, 480, 588]
full_rect:[0, 0, 865, 766]
behavior:none
predicate:
emotion:Anger
episode:1, scene:225, frame:0 st:['00:12:23;18'], et:['00:12:23;18']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[412, 213, 493, 328]
full_rect:[326, 141, 583, 556]
behavior:none
predicate:
emotion:Happiness
episode:1, scene:225, frame:1 st:['00:12:23;18'], et:['00:12:23;18']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[417, 212, 498, 327]
full_rect:[336, 138, 593, 553]
behavior:none
predicate:
emotion:Happiness
episode:1, scene:225, frame:2 st:['00:12:23;18'], et:['00:12:23;18']
place:office
num_of_persons:1
person_id:Haeyoung1
face_rect:[375, 229, 456, 344]
full_rect:[289, 141, 585, 590]
behavio

In [ ]:
{"directed": true, "multigraph": true, 
 "graph": {"name": "G", "node": {"fontsize": "7", "height": "0.1"}, 
           "edge": {"fontsize": "7"}}, 
 "nodes": [{"color": "lightgrey", "style": "filled", "id": "episode_1"}, 
           {"color": "lightgrey", "style": "filled", "id": "scene_10"}, 
           {"color": "lightgrey", "style": "filled", "id": "frame_0"}, 
           {"color": "lightblue", "style": "filled", "id": "kitchen"}, 
           {"color": "lightblue", "style": "filled", "id": "sound"}, 
           {"id": "Jeongsuk"}, 
           {"color": "green", "style": "filled", "id": "cook"}, 
           {"color": "blue", "style": "filled", "id": "Happiness"}, 
           {"id": "Deogi"}], 
 "links": [{"source": "Jeongsuk", "target": "cook", "key": 0}, 
           {"source": "Jeongsuk", "target": "Happiness", "key": 0}, 
           {"source": "Deogi", "target": "cook", "key": 0}, 
           {"source": "Deogi", "target": "Happiness", "key": 0}]
}